In [23]:
import numpy as np
import logging

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [24]:
## parameters for loading data
_num_tuples = 1296
_num_features = 45
_label_dims = 11

In [25]:
def load_data(files, num_tuples, num_features, label_dims):
    data_file, label_file = files
    print(data_file, label_file)
    data = np.zeros((num_tuples, num_features))
    labels = np.zeros((num_tuples, label_dims))
    
    with open(data_file, 'rb') as f:
        for i,line in enumerate(f.readlines()):
            ##################[ISSUE! @YUZHE]HARD CODED, PLEASE CHECK THE DATA FORMAT ########################
            #processed = line.rstrip('\n').rstrip('\r').rstrip(',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,').split(',')
            processed = line.rstrip('\n').split(',')
            ###################################################################################################
            #logger.info(processed)
            try:
                assert num_features == len(processed)
                data[i,:] = processed
            except AssertionError as err:
                logger.info("Wrong Feature Number claimed !, {}, {}".format(num_features, len(processed)))
                
    with open(label_file, 'rb') as f:
        for i,line in enumerate(f.readlines()):
            #processed = line.rstrip('\n').split(',')
            processed = line.rstrip('\n').rstrip('\r').rstrip(',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,').split(',')
            #print(len(processed), label_dims)
            try:
                assert label_dims == len(processed)
                
                labels[i,:] = processed
            except AssertionError as err:
                logger.info("Wrong Label Dimensions claimed !")
        return data, labels

In [26]:
data, labels = load_data(('spectrum.csv', 'temperature.csv'), _num_tuples, _num_features, _label_dims)

('spectrum.csv', 'temperature.csv')


In [27]:
class Dataset(object):
    def __init__(self, data):
        # m, n denote number of tuples and features respectively
        self._m = data[0].shape[0]
        self._n = data[0].shape[1]
        self._training_data = data[0]
        self._training_labels = data[1]
        
    def __len__(self):
        return self._m
    
    def __getitem__(self, idx):
        return  self._training_data[idx,:], self._training_labels[idx,:] 

    def fetch_col(self, col_index):
        return self._training_data[:, col_index]

    def shuffle(self, seed=None):
        if seed:
            np.random.seed(seed=seed)
        shuffled_indices = np.arange(self._m)
        np.random.shuffle(shuffled_indices)
        self._training_data = np.take(self._training_data, shuffled_indices, axis=0)
        self._training_labels = np.take(self._training_labels, shuffled_indices)

    @property
    def num_tuples(self):
        return self._m

    @property
    def num_features(self):
        return self._n

    @property
    def labels(self):
        return self._training_labels

    @property
    def data_table(self):
        return self._training_data

In [28]:
dataset = Dataset((data, labels))

In [29]:
import torch
from torch import nn

# model definition
# notes from Yuzhe: Here are the parameters for the neural network,total of 3 layers of network
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(45, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 11)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
        ## notes from Yuzhe: Following are the activate function for network
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        #x = self.relu(x)
        return x
    def name(self):
        return 'nn'

In [30]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

## notes from Yuzhe, following are parameters that can be changed to modify the training code
args = {'lr':0.001, ## learning rate
        'max_steps':10,   ## maximum numbers of iteration
        'batch_size':100,   
        'epoch':30,
        'enable_gpu':None}
train_loader = DataLoader(dataset, batch_size=args['batch_size'], shuffle=True)

In [31]:
import torch.optim as optim

device = torch.device("cpu")
model = NN().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=0.9)

In [32]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.float().to(device), target.float().to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, target)
        if epoch >= 499:
            print(output, target)
        loss.backward()
        optimizer.step()
        logger.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.mse_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    logger.info('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [33]:
for epoch in range(1, args['epoch'] + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    #test(args, model, device, test_loader)

INFO:root:Train Epoch: 1 [0/1296 (0%)]	Loss: 406806.562500
INFO:root:Train Epoch: 1 [100/1296 (8%)]	Loss: 399810.687500
INFO:root:Train Epoch: 1 [200/1296 (15%)]	Loss: 400475.468750
INFO:root:Train Epoch: 1 [300/1296 (23%)]	Loss: 405205.437500
INFO:root:Train Epoch: 1 [400/1296 (31%)]	Loss: 395530.843750
INFO:root:Train Epoch: 1 [500/1296 (38%)]	Loss: 380604.437500
INFO:root:Train Epoch: 1 [600/1296 (46%)]	Loss: 304379.093750
INFO:root:Train Epoch: 1 [700/1296 (54%)]	Loss: 6551596.000000
INFO:root:Train Epoch: 1 [800/1296 (62%)]	Loss: 404725.531250
INFO:root:Train Epoch: 1 [900/1296 (69%)]	Loss: 398363.593750
INFO:root:Train Epoch: 1 [1000/1296 (77%)]	Loss: 396047.968750
INFO:root:Train Epoch: 1 [1100/1296 (85%)]	Loss: 399431.531250
INFO:root:Train Epoch: 1 [1152/1296 (92%)]	Loss: 398835.468750
INFO:root:Train Epoch: 2 [0/1296 (0%)]	Loss: 385013.656250
INFO:root:Train Epoch: 2 [100/1296 (8%)]	Loss: 381360.718750
INFO:root:Train Epoch: 2 [200/1296 (15%)]	Loss: 366773.093750
INFO:root:Tr

INFO:root:Train Epoch: 11 [600/1296 (46%)]	Loss: 3895.736816
INFO:root:Train Epoch: 11 [700/1296 (54%)]	Loss: 2741.863525
INFO:root:Train Epoch: 11 [800/1296 (62%)]	Loss: 2995.912109
INFO:root:Train Epoch: 11 [900/1296 (69%)]	Loss: 3453.706787
INFO:root:Train Epoch: 11 [1000/1296 (77%)]	Loss: 3405.600098
INFO:root:Train Epoch: 11 [1100/1296 (85%)]	Loss: 2868.036621
INFO:root:Train Epoch: 11 [1152/1296 (92%)]	Loss: 3323.678711
INFO:root:Train Epoch: 12 [0/1296 (0%)]	Loss: 3726.866211
INFO:root:Train Epoch: 12 [100/1296 (8%)]	Loss: 2769.741943
INFO:root:Train Epoch: 12 [200/1296 (15%)]	Loss: 3299.605713
INFO:root:Train Epoch: 12 [300/1296 (23%)]	Loss: 3401.320312
INFO:root:Train Epoch: 12 [400/1296 (31%)]	Loss: 2912.167236
INFO:root:Train Epoch: 12 [500/1296 (38%)]	Loss: 2871.958740
INFO:root:Train Epoch: 12 [600/1296 (46%)]	Loss: 3188.794678
INFO:root:Train Epoch: 12 [700/1296 (54%)]	Loss: 2934.572021
INFO:root:Train Epoch: 12 [800/1296 (62%)]	Loss: 2976.453125
INFO:root:Train Epoch: 12

INFO:root:Train Epoch: 21 [1100/1296 (85%)]	Loss: 2753.583740
INFO:root:Train Epoch: 21 [1152/1296 (92%)]	Loss: 3260.686035
INFO:root:Train Epoch: 22 [0/1296 (0%)]	Loss: 3060.831299
INFO:root:Train Epoch: 22 [100/1296 (8%)]	Loss: 3295.530273
INFO:root:Train Epoch: 22 [200/1296 (15%)]	Loss: 3594.310303
INFO:root:Train Epoch: 22 [300/1296 (23%)]	Loss: 2934.298828
INFO:root:Train Epoch: 22 [400/1296 (31%)]	Loss: 2884.571533
INFO:root:Train Epoch: 22 [500/1296 (38%)]	Loss: 3035.305420
INFO:root:Train Epoch: 22 [600/1296 (46%)]	Loss: 2870.808350
INFO:root:Train Epoch: 22 [700/1296 (54%)]	Loss: 3092.178223
INFO:root:Train Epoch: 22 [800/1296 (62%)]	Loss: 2983.266113
INFO:root:Train Epoch: 22 [900/1296 (69%)]	Loss: 2920.132324
INFO:root:Train Epoch: 22 [1000/1296 (77%)]	Loss: 3114.683105
INFO:root:Train Epoch: 22 [1100/1296 (85%)]	Loss: 2972.305664
INFO:root:Train Epoch: 22 [1152/1296 (92%)]	Loss: 2899.571045
INFO:root:Train Epoch: 23 [0/1296 (0%)]	Loss: 2962.298828
INFO:root:Train Epoch: 23 

In [23]:
for p in model.parameters():
    print(p)

Parameter containing:
tensor([[-0.0071, -0.1287,  0.1259,  ...,  0.1253, -0.0517,  0.0411],
        [-0.1447, -0.1280, -0.0043,  ..., -0.1127, -0.0791, -0.0833],
        [-0.0141, -0.0029,  0.1197,  ..., -0.0019,  0.0951,  0.0563],
        ...,
        [-0.0902, -0.0325, -0.0210,  ..., -0.0407, -0.0002,  0.1448],
        [ 0.0771,  0.1247,  0.0349,  ...,  0.0376,  0.0866,  0.0797],
        [ 0.1361,  0.0082, -0.0669,  ..., -0.0285,  0.1064, -0.1164]],
       requires_grad=True)
Parameter containing:
tensor([-8.5227e+02, -9.3861e-02, -5.5554e-02, -4.0435e-01, -4.3156e-02,
        -7.7015e-02, -1.0699e+04, -4.7856e+03, -6.8719e-02, -2.5835e-02,
        -6.5731e-02, -2.0890e+04, -6.6544e-03, -6.2132e-03, -7.8425e-02,
        -3.8066e+03, -1.5754e+04, -1.2599e+04, -5.2499e+03, -1.8008e+04,
        -4.2330e-03, -1.4033e-01, -2.2787e+04, -1.4855e-01, -1.4133e-01,
        -2.0877e+03, -1.0252e-01, -8.6479e+03, -7.9017e-02, -1.7272e+04,
        -5.5592e-01, -9.5456e-02, -5.3845e-02, -3.0728e+0